# 8.3 更に大規模なデータの処理：オンラインアルゴリズムとアウトオブコア学習

- 多くの現実のアプリケーションでは、コンピュータのメモリに収まらないほど大規模なデータセットを処理することも珍しくない。

- 誰もがスーパーコンピュータを利用できるわけではないため、ここではそうした大規模なデータセットの処理を可能にするアウトオブコア学習という手法を適用する。

- ここでは、scikit-learnのSGDClassifierクラスのpartial_fitメソッドを使用することで、ローカルドライブからドキュメントを直接ストリーミングし、ドキュメントの小さなミニバッチを使って、ロジスティック回帰モデルをトレーニングする。

In [1]:
import numpy as np
import re
from nltk.corpus import stopwords
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>','',text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text.lower())
    text = re.sub('[\W]+',' ',text.lower()) + ' '.join(emoticons).replace('-','')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

- ジェネレータ関数stream_docsを定義する。この関数は、ドキュメントを１つずつ読み込んで返す。

In [17]:
import codecs
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) #ヘッダーを読み飛ばす
        for line in csv:
            text, label = line[:-3],int(line[-2])
            yield text, label

In [18]:
stream_docs(path='./movie_data.csv').next()

('"I went to the cinema to watch a preview of this film without knowing anything about it. Recognizing Jennifer Lynch\'s name and seeing the 18 certificate I realised it might be disturbing. In actuality I found the film a farce. I found myself giggling in disbelief through parts of it. The acting is atrocious- Bill Pullman and his ridiculous twitching face. I do almost pity the actors though as the script offers them no chance of any believable character interaction. After some shocking incident, (there is plenty to ""try"" and shock the viewer in this film), 2 characters are seen sharing a beer and talking about the weather. Everything was overstated, or thought it was being clever when really it was obvious! The performance from the little girl character named Stephanie was the best thing about the film. Quiet and intense. I really could not recommend this film to anyone. Its violent without point, ridiculous characters, bad acting, bad script and plain silly."',
 0)

- get_minibatch関数を定義する。この関数は、stream_docs関数からドキュメントストリームを受け取り、size引数によって指定された個数のドキュメントを返す。

In [19]:
def get_minibatch(doc_stream,size):
    docs, y = [],[]
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs,y

- アウトオブコア学習にCountVectorizerは使用できない。アウトオブコア学習では、語彙が完全にメモリに読み込まれていることが要求される。
- TfidfVectorizerは、逆文書頻度を計算するにあたって、トレーニングデータセットの特徴ベクトルがすべてメモリに読み込まれていることを要求する。
- scikit-learnには、HashingVectoriaerという便利なベクトライザが実装されている。こいつはデータに依存しない。

In [20]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
vect = HashingVectorizer(decode_error ='ignore',
                        n_features=2**21, #特徴量が^21
                        preprocessor=None,
                        tokenizer=tokenizer)
clf = SGDClassifier(loss='log', #ロジスティック回帰分類器で初期化
                    random_state=1,
                    n_iter=1)
doc_stream = stream_docs(path='./movie_data.csv')

- HashingVectorizerで特徴量の個数に大きな値を設定すると、ハッシュの衝突が発生する可能性は少なくなるが、ロジスティック回帰モデルの係数の値が大きくなることに注意。

In [21]:
import pyprind
pbar = pyprind.ProgBar(45)
classes = np.array([0,1])
for _ in range(45):
    X_train,y_train = get_minibatch(doc_stream,size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train,y_train,classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:22


- forループでドキュメントの45個のミニバッチを処理している。ミニバッチはそれぞれ1，000個のドキュメントを構成する。

In [23]:
#最後の5，000個のドキュメントを使ってモデルの性能を評価する。

In [24]:
X_test,y_test = get_minibatch(doc_stream,size=5000)
X_test = vect.transform(X_test)
print('Accuracy: %.3f') % clf.score(X_test,y_test)

Accuracy: 0.874


- アウトオブコア学習はメモリ効率が非常によく、完了が早い。
- 最後の5，000個のドキュメントを使ってモデルを更新できる。

In [25]:
clf = clf.partial_fit(X_test,y_test)